In [1]:
import os
import openai
import sys
sys.path.append('../..')
import utils

import panel as pn  # GUI
pn.extension()

openai.api_key = 'please use your key'

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"

    # Step 1: Moderation API check
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    # Step 2: Extract category and product details
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    category_and_product_list = utils.read_string_to_list(category_and_product_response)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: Look up product information
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Create system and user messages for response
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug: print("Step 4: Generated response to user question.")
    
    all_messages += messages[1:]  # Append user and assistant messages to context

    # Step 5: Moderation API check for the response
    response = openai.Moderation.create(input=final_response)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Model evaluation of the response
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: Check if the model confirms the response
    if "Y" in evaluation_response:
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

# Function to collect user input and update conversation panel
def collect_messages(event=None, debug=False):
    user_input = inp.value
    if debug: print(f"User Input = {user_input}")
    if not user_input:
        return
    inp.value = ''
    
    global context
    response, context = process_user_message(user_input, context, debug=debug)

    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600))
    )
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'}))
    )
    
    return pn.Column(*panels)

# Initialize the dashboard components
panels = []  # Collect display panels for conversation
context = [{'role': 'system', 'content': "You are a Service Assistant"}]  # Initial context for conversation

inp = pn.widgets.TextInput(placeholder='Enter text here...')
button_conversation = pn.widgets.Button(name="Service Assistant")

# Bind the button click to the collect_messages function
button_conversation.on_click(collect_messages)

# Create the dashboard layout
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(pn.bind(collect_messages, button_conversation), loading_indicator=True, height=300)
)

dashboard

Traceback (most recent call last):
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\panel\viewable.py", line 477, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 104, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\document\callbacks.py", line 453, in invoke_with_curdoc
 return f()
 ^^^
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 104, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\document\document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_event(self, event, setter)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\document\events.py", line 244, in handle_event
 event_cls._handle_event(doc, event)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\document\events.py", line 279, in _handle_event
 cb(event.msg_data)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\document\callbacks.py", line 400, in trigger_event
 model._trigger_event(event)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\util\callback_manager.py", line 111, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\document\callbacks.py", line 262, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\document\callbacks.py", line 453, in invoke_with_curdoc
 return f()
 ^^^
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\bokeh\util\callback_manager.py", line 107, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\panel\reactive.py", line 559, in _comm_event
 state._handle_exception(e)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\panel\io\state.py", line 468, in _handle_exception
 raise exception
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\panel\reactive.py", line 557, in _comm_event
 self._process_bokeh_event(doc, event)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\panel\reactive.py", line 494, in _process_bokeh_event
 self._process_event(event)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\panel\widgets\button.py", line 241, in _process_event
 self.param.trigger('value')
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\param\parameterized.py", line 2474, in trigger
 self_.update(dict(params, **triggers))
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\param\parameterized.py", line 2319, in update
 restore = dict(self_._update(arg, **kwargs))
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\param\parameterized.py", line 2352, in _update
 self_._batch_call_watchers()
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\param\parameterized.py", line 2546, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\param\parameterized.py", line 2506, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\panel\param.py", line 891, in _replace_pane
 new_object = self.eval(self.object)
 ^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ELCOT\anaconda3\envs\aidata\Lib\site-packages\panel\param.py", line 1106, in eval
 return eval_function_with_deps(ref)
 ^

Column
    [0] TextInput(placeholder='Enter text here...')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)